##Setting up Keras And loading Data And Libraries

In [1]:
pip install keras==1.2.2

     |████████████████████████████████| 184kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/55/07/cf/b32db0a8d243b2fd6759d5d7cb650aa20670b2b740209cbf7e
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 1.2.2 which is incompatible.
ERROR: kapre 0.1.3.1 has requirement keras>=2.0.0, but you'll have keras 1.2.2 which is incompatible.
ERROR: fancyimpute 0.4.3 has requirement keras>=2.0.0, but you'll have keras 1.2.2 which is incompatible.
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [2]:
pip install np_utils

In [0]:
from keras.applications.vgg16 import VGG16
import cv2
import numpy as np
from keras.models import model_from_json
#from keras.utils.vis_utils import plot_model
from keras.applications.vgg16 import preprocess_input
import os
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import load_model

##Loading the Data

In [3]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/


In [0]:
path='/gdrive/My Drive/Colab Notebooks/VQA_DATASET/Final_Dataset1.json'

In [0]:
data = json.load(open(path))
train_data = pd.DataFrame(data)

In [8]:
train_data.head()

,ans,ques_id,question,img_id,image_name,image_feature
0,5218,25000,what is in front of the giraffes,25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
1,1887,25001,what do these giraffes have in common,25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
10,2993,25010,what is on the ground next to the giraffe on t...,25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
100,3363,149001,is the sky clear,149,COCO_train2014_000000000149.jpg,"[3.93909e-05, 1.6657e-05, 9.6804e-06, 3.1817e-..."
1000,4219,1522001,what kind of place is this,1522,COCO_train2014_000000001522.jpg,"[9.9e-09, 3.589e-07, 9.6e-09, 7e-10, 5.8e-09, ..."


##Tokenizing the Data

In [9]:
# Tokenizing
t = Tokenizer()
t.fit_on_texts(train_data['question'])
vocab_size = len(t.word_index) + 1
print(t.word_index)
# integer encode the documents

train_data['question'] = t.texts_to_sequences(train_data['question'])
print(vocab_size)


{'the': 1, 'is': 2, 'what': 3, 'are': 4, 'this': 5, 'in': 6, 'on': 7, 'of': 8, 'a': 9, 'how': 10, 'many': 11, 'color': 12, 'there': 13, 'does': 14, 'man': 15, 'people': 16, 'picture': 17, 'to': 18, 'wearing': 19, 'where': 20, 'it': 21, 'these': 22, 'have': 23, 'photo': 24, 'or': 25, 'kind': 26, 'person': 27, 'do': 28, 'you': 29, 'doing': 30, 'type': 31, 'woman': 32, 'animal': 33, 'be': 34, 'holding': 35, 'they': 36, 'animals': 37, 'room': 38, 'for': 39, 'sign': 40, 'can': 41, 'train': 42, 'at': 43, 'his': 44, 'dog': 45, 'shirt': 46, 'food': 47, 'cat': 48, 'water': 49, 'which': 50, 'he': 51, 'that': 52, 'any': 53, 'bus': 54, 'see': 55, 'an': 56, 'made': 57, 'plate': 58, 'mans': 59, 'sitting': 60, 'why': 61, 'playing': 62, 'time': 63, 'shown': 64, 'taken': 65, 'table': 66, 'left': 67, 'with': 68, 'background': 69, 'right': 70, 'plane': 71, 'was': 72, 'sport': 73, 'number': 74, 'standing': 75, 'look': 76, 'pizza': 77, 'from': 78, 'street': 79, 'who': 80, 'being': 81, 'white': 82, 'boy': 8

In [10]:
train_data.head()

,ans,ques_id,question,img_id,image_name,image_feature
0,5218,25000,"[3, 2, 6, 113, 8, 1, 129]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
1,1887,25001,"[3, 28, 22, 129, 23, 6, 996]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
10,2993,25010,"[3, 2, 7, 1, 93, 172, 18, 1, 145, 7, 1, 70]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
100,3363,149001,"[2, 1, 88, 313]",149,COCO_train2014_000000000149.jpg,"[3.93909e-05, 1.6657e-05, 9.6804e-06, 3.1817e-..."
1000,4219,1522001,"[3, 26, 8, 201, 2, 5]",1522,COCO_train2014_000000001522.jpg,"[9.9e-09, 3.589e-07, 9.6e-09, 7e-10, 5.8e-09, ..."


##Splitting The Data into Train_Test

In [0]:
x = train_data[['ques_id','question','image_feature']]
y = train_data['ans']

In [12]:
train_data.head()

,ans,ques_id,question,img_id,image_name,image_feature
0,5218,25000,"[3, 2, 6, 113, 8, 1, 129]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
1,1887,25001,"[3, 28, 22, 129, 23, 6, 996]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
10,2993,25010,"[3, 2, 7, 1, 93, 172, 18, 1, 145, 7, 1, 70]",25,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
100,3363,149001,"[2, 1, 88, 313]",149,COCO_train2014_000000000149.jpg,"[3.93909e-05, 1.6657e-05, 9.6804e-06, 3.1817e-..."
1000,4219,1522001,"[3, 26, 8, 201, 2, 5]",1522,COCO_train2014_000000001522.jpg,"[9.9e-09, 3.589e-07, 9.6e-09, 7e-10, 5.8e-09, ..."


In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state= 180)

## GLOVE Conversion

In [14]:
!unzip "/gdrive/My Drive/Colab Notebooks/VQA_DATASET/glove.6B.300d.zip"

Archive:  /gdrive/My Drive/Colab Notebooks/VQA_DATASET/glove.6B.300d.zip
  inflating: glove.6B.300d.txt       


In [15]:
def encode_question(imgs):
    max_length = 26
    N = len(imgs)
    print(N)
    print(type(imgs))
    label_arrays = np.zeros((N, max_length), dtype='uint32')
    #print(label_arrays)
    label_length = np.zeros(N, dtype='uint32')
    #print("LL",label_length)
    question_id = np.zeros(N, dtype='uint32')
   # print("QI",question_id)
    question_counter = 0
    for i in range(len(imgs)):
#         print(type(img))
        img = imgs.iloc[i]
        question_id[question_counter] = img['ques_id']
        label_length[question_counter] = min(max_length, len(img['question'])) # record the length of this sequence
        question_counter += 1
        for k,w in enumerate(img['question']):
            if k < max_length:
                #print(f'i = {i}, k = {k}, w = {w}')
                label_arrays[i,k] = w
    return label_arrays,label_length,question_id                         
ques_train, ques_length_train, question_id_train = encode_question(X_train)
ques_test, ques_length_test, question_id_test = encode_question(X_test)

37328
<class 'pandas.core.frame.DataFrame'>
15998
<class 'pandas.core.frame.DataFrame'>


In [0]:
def right_align(seq,lengths):
    v = np.zeros(np.shape(seq))
    N = np.shape(seq)[1]
    for i in range(np.shape(seq)[0]):
        v[i][N-lengths[i]:N]=seq[i][0:lengths[i]]
    return v

In [0]:
ques_train = right_align(ques_train, ques_length_train)
ques_test = right_align(ques_test, ques_length_test)

In [19]:
#Glove Embeedings
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
#Creating the Weighted Matrix
embedding_matrix = np.zeros((vocab_size, 300))
for word, index in t.word_index.items():
    if index > vocab_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [21]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.04656     0.21318001 -0.0074364  ...  0.0090611  -0.20988999
   0.053913  ]
 [-0.1749      0.22956     0.24924    ... -0.24131    -0.40402001
   0.054744  ]
 ...
 [-0.28329     0.11537     0.23277999 ... -0.20263     0.63609999
  -0.72575998]
 [-0.57538998  0.012679   -0.30335999 ...  0.25523001 -0.20947
  -0.17614999]
 [ 0.41062    -0.26291999  0.073937   ... -0.23774     0.40763
   0.47595   ]]


In [22]:
X_train.head()

,ques_id,question,image_feature
45930,69564002,"[10, 11, 6315, 4, 6, 1, 69, 112]","[1e-09, 3.742e-07, 1.4e-09, 1.52e-08, 2e-10, 9..."
6908,10877001,"[2, 1, 32, 1630]","[9.2e-09, 2.7238e-06, 2.7502e-06, 6.592e-07, 7..."
37663,57965001,"[3, 105, 2, 1, 90, 7]","[3.9178e-06, 8.379e-06, 9.2794e-06, 8.4208e-06..."
18228,48339012,"[41, 29, 55, 9, 4760, 464, 1, 385]","[0.0001167302, 0.0004371512, 0.0010246518, 0.0..."
27834,42867001,"[4, 16, 463, 1, 257]","[4.116e-07, 5.42085e-05, 2.5277e-06, 1.6203e-0..."






## Preparing Data

In [0]:
train=pd.DataFrame(X_train)
train['ans']=y_train

In [0]:
dict_img_train={}
#dict_qu={}
target_train=[]

for i in train['ques_id']:
  dict_img_train[i]=train[train['ques_id']==i]['image_feature'][0]
  target_train.append(train[train['ques_id']==i]['ans'][0])

In [0]:
test=pd.DataFrame(X_test)
test['ans']=y_test

In [0]:
dict_img_test={}
#dict_qu={}
target_test=[]

for i in test['ques_id']:
  dict_img_test[i]=test[test['ques_id']==i]['image_feature'][0]
  target_test.append(test[test['ques_id']==i]['ans'][0])

In [27]:
x_train=np.array(list(dict_img_train.values()))
x_train.shape

(37328, 1000)

In [28]:
x_test=np.array(list(dict_img_test.values()))
x_test.shape

(15998, 1000)

In [0]:
x_qu_train= ques_train
x_qu_test= ques_test

In [30]:
print(x_qu_train.shape)
print(x_qu_test.shape)

(37328, 26)
(15998, 26)


In [31]:
y_tr=np.array(target_train)
y_train=to_categorical(y_train)
y_train.shape

(37328, 5666)

In [32]:
y_te=np.array(target_test)
y_test=to_categorical(y_te)
y_test.shape

(15998, 5666)

## **Image** **Model**

In [33]:
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential,Model
from keras.layers import Activation, Dense,Flatten,Input
from keras.layers.core import  Dropout,Reshape
image_feature_size=1000

model_image = Sequential()

model_image.add(Dense(2048,input_shape=(1000,)))
model_image.add(Activation('tanh'))
model_image.add(Dropout(0.5))
print(model_image.summary())

W0730 17:00:59.894577 140344025024384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:321: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 17:00:59.947551 140344025024384 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:634: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 17:00:59.967365 140344025024384 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:491: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 2048)          2050048     dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2048)          0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 2048)          0           activation_1[0][0]               
Total params: 2,050,048
Trainable params: 2,050,048
Non-trainable params: 0
____________________________________________________________________________________________________
None


## Language Model

In [34]:
from keras.layers import LSTM, Dense, Embedding 
embedding_dim               = 300
number_of_LSTM              = 3
number_of_hidden_units_LSTM = 64
max_length_questions        = 26
number_of_dense_layers      = 3
number_of_hidden_units      = 2048
activation_function         = 'tanh'
dropout_pct                 = 0.5


model_language = Sequential()
model_language.add(Embedding(vocab_size, embedding_dim ,weights = [embedding_matrix],input_length =max_length_questions , trainable = False))
model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=True))
model_language.add(Dropout(dropout_pct))
model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=False))
model_language.add(Dropout(dropout_pct))
model_language.add(Dense(number_of_hidden_units))
model_language.add(Activation(activation_function))
print(model_language.summary())

W0730 17:01:05.275204 140344025024384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:112: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0730 17:01:05.276441 140344025024384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:117: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0730 17:01:05.277939 140344025024384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:122: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0730 17:01:05.708370 140344025024384 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1029: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructio

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 26, 300)       2119800     embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 26, 64)        93440       embedding_1[0][0]                
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 26, 64)        0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 64)            33024       dropout_2[0][0]                  
___________________________________________________________________________________________

## Combine Model

In [35]:
from keras.engine import Merge
model = Sequential()
model.add(Merge([model_image,model_language], mode='mul'))
print('merged')
#model.add(Dropout(0.5))
model.add(Dense(1024, activation='tanh'))
model.add(Dense(1000, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(5666, activation='softmax'))  
print(model.summary())

merged
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 2048)          2050048     dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2048)          0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 2048)          0           activation_1[0][0]               
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 26, 300)       2119800     embedding_input_1[0][0]          
____________________________________________________________________________________

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0730 17:01:21.894418 140344025024384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:658: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
#y=np.array(train_data['ans'])
x_trains=[x_train,x_qu_train]

In [0]:
x_tests=[x_test,x_qu_test]

In [39]:
history = model.fit(x_trains,y_train,batch_size = 500 ,nb_epoch = 40,validation_data=(x_tests,y_test))

W0730 17:01:35.124335 140344025024384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 37328 samples, validate on 15998 samples
Epoch 1/40
37328/37328 [==============================] - 65s - loss: 5.6468 - acc: 0.1818 - val_loss: 4.7522 - val_acc: 0.1822
Epoch 2/40
37328/37328 [==============================] - 63s - loss: 4.6192 - acc: 0.1821 - val_loss: 4.4845 - val_acc: 0.1819
Epoch 3/40
37328/37328 [==============================] - 62s - loss: 4.2893 - acc: 0.2088 - val_loss: 4.1568 - val_acc: 0.2278
Epoch 4/40
37328/37328 [==============================] - 61s - loss: 4.0374 - acc: 0.2280 - val_loss: 4.0341 - val_acc: 0.2381
Epoch 5/40
37328/37328 [==============================] - 61s - loss: 3.8943 - acc: 0.2315 - val_loss: 3.9855 - val_acc: 0.2350
Epoch 6/40
37328/37328 [==============================] - 62s - loss: 3.7854 - acc: 0.2405 - val_loss: 3.8687 - val_acc: 0.2378
Epoch 7/40
37328/37328 [==============================] - 61s - loss: 3.6927 - acc: 0.2411 - val_loss: 3.8476 - val_acc: 0.2470
Epoch 8/40
37328/37328 [==============================

##Saving the Model

In [0]:
model.save('VQA_test_model.h5')